In [3]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# Change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name
username = "aacuser"
password = "SNHU1234"
host = 'nv-desktop-services.apporto.com'
port = '32397'
db_name = 'AAC'
collection_name ='animals'

# Connect to database via CRUD Module
db = AnimalShelter(username, password, host, port, db_name, collection_name)

# class read method must support return of list object and accept projection JSON input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'], inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png'  # Replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
# FIX ME Also remember to include a unique identifier such as your name or date
# html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Center(
        html.Div([
            html.Img(
                src='data:image/png;base64,{}'.format(encoded_image.decode()),
                style={'height': '200px', 'margin-bottom': '10px'}
            ),
            html.Div('Essra Alaudi', style={'font-size': '14px', 'text-align': 'center'})
        ])
    ),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),
    html.Div([
        # FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        html.Label('Rescue Type:'),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'Mountain/Wilderness Rescue'},
                {'label': 'Disaster/Individual Tracking', 'value': 'Disaster/Individual Tracking'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Reset',
            labelStyle={'display': 'block'}
        ),
        html.Br(),

        # Dropdown for breed filter (multi-select)
        html.Label('Breed:'),
        dcc.Dropdown(
            id='breed-filter',
            options=[{'label': breed, 'value': breed} for breed in df['breed'].unique()],
            placeholder='Select Preferred Breeds',
            multi=True
        ),
        html.Br(),

        # Dropdown for sex filter (multi-select)
        html.Label('Sex Upon Outcome:'),
        dcc.Dropdown(
            id='sex-filter',
            options=[
                {'label': 'Intact Male', 'value': 'Intact Male'},
                {'label': 'Intact Female', 'value': 'Intact Female'},
                {'label': 'Neutered Male', 'value': 'Neutered Male'},
                {'label': 'Spayed Female', 'value': 'Spayed Female'},
                {'label': 'Unknown', 'value': 'Unknown'}
            ],
            placeholder='Select Preferred Sex',
            multi=True
        ),
        html.Br(),

        # Dropdown for age range (multi-select)
        html.Label("Filter by Age (in weeks)"),
        dcc.RangeSlider(
            id='age-filter',
            min=20,  # Start of the preferred age range
            max=300,  # End of the preferred age range
            step=1,
            value=[20, 300],  # Default range
            marks={i: str(i) for i in range(20, 301, 10) if i in [20, 26, 50, 100, 150, 200, 250, 300]},
        ),
    ]),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        # FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        #If you completed the Module Six Assignment, you can copy in the code you created here 
        row_selectable="single",  # Single-row selection
        selected_rows=[0],  # Default selection for map callback
        style_table={'height': '500px', 'overflowY': 'auto'},  # Scrollable table
        style_cell={'textAlign': 'left'},  # Align text in the table to the left
        page_size=10,  # Pagination with 10 rows per page
        sort_action='native',  # Client-side sorting
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 html.Div(
                     id='graph-id',
                     className='col s12 m6',
                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                 )
             ])
])

#############################################
# Interaction Between Components / Controller
#############################################
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value'),
     Input('breed-filter', 'value'),
     Input('sex-filter', 'value'),
     Input('age-filter', 'value'),
     ])
def update_dashboard(filter_type, breeds, sex, age_values):
    # Base query dictionary
    query = {}

    # Rescue type-based filtering
    if filter_type and filter_type != 'Reset':
        if filter_type == 'Water Rescue':
            query['breed'] = {'$in': [
                'Labrador Retriever Mix',
                'Chesapeake Bay Retriever',
                'Newfoundland'
            ]}
            query['sex_upon_outcome'] = 'Intact Female'
            query['age_upon_outcome_in_weeks'] = {'$gte': 26, '$lte': 156}
        elif filter_type == 'Mountain/Wilderness Rescue':
            query['breed'] = {'$in': [
                'German Shepherd',
                'Alaskan Malamute',
                'Old English Sheepdog',
                'Siberian Husky',
                'Rottweiler'
            ]}
            query['sex_upon_outcome'] = 'Intact Male'
            query['age_upon_outcome_in_weeks'] = {'$gte': 26, '$lte': 156}
        elif filter_type == 'Disaster/Individual Tracking':
            query['breed'] = {'$in': [
                'Doberman Pinscher',
                'German Shepherd',
                'Golden Retriever',
                'Bloodhound',
                'Rottweiler'
            ]}
            query['sex_upon_outcome'] = 'Intact Male'
            query['age_upon_outcome_in_weeks'] = {'$gte': 20, '$lte': 300}

    # Additional manual filters
    # Breed filter (multi-selection)
    if breeds:
        if 'breed' not in query or not isinstance(query['breed'], dict):
             query['breed'] = {'$in': breeds}
        else:
            # Merge the existing filter with the new condition
            query['breed']['$in'] = list(
                set(query['breed']['$in']).intersection(breeds)
        )

    # Sex filter (multi-selection)
    if sex:
        if 'sex_upon_outcome' not in query or not isinstance(query['sex_upon_outcome'], dict):
            query['sex_upon_outcome'] = {'$in': sex}
        else:
            # Merge the existing filter with the new condition
            query['sex_upon_outcome']['$in'] = list(
                set(query['sex_upon_outcome']['$in']).intersection(sex)
        )

    # Age range filter (multi-selection)
    if age_values:
        min_age = min(age_values)
        max_age = max(age_values)
        age_filter = {'$gte': min_age, '$lte': max_age}
        query['age_upon_outcome_in_weeks'] = age_filter if 'age_upon_outcome_in_weeks' not in query else {
            '$gte': max(query['age_upon_outcome_in_weeks']['$gte'], age_filter['$gte']),
            '$lte': min(query['age_upon_outcome_in_weeks']['$lte'], age_filter['$lte'])
        }

    # Query MongoDB and update DataTable
    filtered_data = pd.DataFrame.from_records(db.read(query))
    if '_id' in filtered_data.columns:
        filtered_data.drop(columns=['_id'], inplace=True)

    return filtered_data.to_dict('records')

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    # Check if the input data is None or empty
    if not viewData:
        return "No data available for visualization"

    # Convert the data to a pandas DataFrame
    df_view = pd.DataFrame(viewData)

    # Check if the 'breed' column exists
    if 'breed' not in df_view.columns:
        return "No breed column available for visualization"

    # Check if the DataFrame is empty
    if df_view.empty:
        return "No data available for visualization"

    # Create the pie chart
    return [
        dcc.Graph(
            figure=px.pie(df_view, names='breed', title='Preferred Animals by Breed')
        )
    ]

#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if not selected_columns:
        # If no columns are selected, return an empty style
        return []

    # Return styles for selected columns
    return [
        {
            'if': {'column_id': i},
            'background_color': '#D2F3FF'
        } for i in selected_columns
    ]

# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    # Check if viewData is empty or None
    if not viewData:
        return "No data available for mapping."

    # Convert viewData to a DataFrame
    dff = pd.DataFrame.from_dict(viewData)

    # Handle the selected index (default to the first row if no selection)
    if not index or len(index) == 0:
        row = 0
    else:
        row = index[0]

    # Check if the row index is within bounds
    if row >= len(dff):
        return "Selected row is out of range."

    # Validate latitude and longitude columns
    try:
        lat = dff.iloc[row, 13]  # Assuming column index 13 is latitude
        lon = dff.iloc[row, 14]  # Assuming column index 14 is longitude
    except IndexError:
        return "Latitude and longitude columns are missing or out of range."

    # Ensure latitude and longitude are numeric
    if pd.isna(lat) or pd.isna(lon) or not isinstance(lat, (int, float)) or not isinstance(lon, (int, float)):
        return "Invalid latitude or longitude values."

    # Extract breed and animal name for display
    breed = dff.iloc[row, 4] if 4 < len(dff.columns) else "Unknown Breed"
    animal_name = dff.iloc[row, 9] if 9 < len(dff.columns) else "Unknown Name"

    # Generate the map with a marker
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[lat, lon], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(breed),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(animal_name)
                ])
            ])
        ])
    ]

app.run_server(debug=True)


Dash app running on http://127.0.0.1:21502/
